In [1]:
import torch
import numpy as np
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torchvision
import torch.nn as nn

In [55]:
def squash(s, dim=-1, epsilon=1e-3):
    norm_s = torch.sqrt(torch.sum(t**2, dim, keepdim=True) + epsilon)
    v = (norm_s / (1 + norm_s)) + (s / norm_s)
    return v

In [ ]:
class CapsuleLayer(nn.Module):
    def __init__(self, in_channels, capsule_dimension, num_capsules, kernel_size, routing=False, num_iterations=0, stride=1):
        
        super(CapsuleLayer, self).__init__()
        
        self.in_channels = in_channels
        self.capsule_width = capsule_width
        self.capsule_dimension = capsule_dimension
        self.num_capsules = num_capsules
        self.kernel_size = kernel_size
        self.routing = routing
        self.num_iterations = num_iterations
        self.stride = stride
        
        if not self.routing:
            
            self.width = 6 # (W - F + 2 * P) / S + 1
            
            self.conv = nn.Conv2d(self.in_channels, self.capsule_dimension * self.num_capsules,\
                                  self.kernel_size, self.stride)
            
            self.conv2 = self.conv.view(-1, self.width * self.width * self.num_capsules, self.capsule_dimension)
            
            self.squash_conv = squash(self.conv2)
        
        else:
            
            

        
    def forward(self, x):
        pass
                
                
                
            
        

In [ ]:
class DigitalCapsule(nn.Module):
    def __init__(self, input_size, vector_size, output_size):
        
        super(DigitalCapsule, self).__init__()
        
        self.input_size = input_size
        self.vector_size = vector_size
        self.output_size = output_size
        
        self.vec = None
        
    def forward(self, x):
        

In [ ]:
class Net(nn.Module):
    def __init__(self):
        
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 256, 9)
        
        self.primary_capsule = CapsuleLayer()
        
        self.digital_capsule = Digi
        
    def forward(self, x):
        

In [45]:
t = torch.Tensor([[1, 2, 3], [4, 5, 6], [1, 2, 7]])
print(t.size())
# torch.Size([2, 3, 2])
# print(t.view(-1, 6).size())
# torch.Size([2, 12])

torch.Size([3, 3])


In [46]:
t


 1  2  3
 4  5  6
 1  2  7
[torch.FloatTensor of size 3x3]

In [54]:
torch.sqrt(torch.sum(t**2, dim=-1, keepdim=True) + 0.1)


 3.7550
 8.7807
 7.3553
[torch.FloatTensor of size 3x1]